# Using Keras to classify German Traffic Sign database

## Objectives
- Use the same architecture as in 2.6 but using keras
- Reuse the Data Input and Processing

In [1]:
import tensorflow as tf
import keras

import numpy as np
import cv2
import pickle

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import shuffle

Using TensorFlow backend.


In [2]:
tf.__version__

'1.0.0'

In [3]:
keras.__version__

'1.2.2'

<img align="left" src="files/old.png"> **Load data**

In [4]:
training_file = './data/train2.p'
testing_file = './data/test2.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)

<img align="left" src="files/old.png"> **Standardize naming** 

In [5]:
xtr = train['features']
ytr = train['labels']

xva, xte, yva, yte = train_test_split(test['features'], test['labels'], test_size=0.5)

<img align="left" src="files/old.png"> **One hot encoding**

In [6]:
encoder = LabelBinarizer()
encoder.fit(ytr)
ytr = encoder.transform(ytr).astype(float)
yte = encoder.transform(yte).astype(float)
yva = encoder.transform(yva).astype(float)

<img align="left" src="files/new.png"> **Import keras libraries**

In [7]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Lambda
from keras.utils import np_utils

<img align="left" src="files/new.png"> **Define keras model**

In [8]:
model = Sequential()

model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(32,32,3)))

model.add(Convolution2D(32, 5, 5, activation='relu', border_mode='same'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Convolution2D(64, 5, 5, activation='relu', border_mode='same'))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(43, activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

<img align="left" src="files/new.png"> **Train**

In [9]:
model.fit(xtr, ytr, batch_size=64, nb_epoch=30, verbose=0)

In [10]:
model.metrics_names

['loss', 'acc']

In [11]:
print model.evaluate(xte, yte, verbose=0)

[0.40607819056683581, 0.95518606497197522]


In [12]:
print model.evaluate(xva, yva, verbose=0)

[0.41653184816695837, 0.95740300870942197]
